In [ ]:
import pandas as pd


In [ ]:
train = pd.read_csv('train.csv',sep=',')

In [ ]:
train

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...,...
497814,497815,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497815,497816,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497816,497817,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0
497817,497818,"Shanghai Kechuan Trading Co., Ltd.",Shanghai M&G Stationery Inc.,0


In [ ]:
test = pd.read_csv('test.csv',sep=',')

In [ ]:
test

,pair_id,name_1,name_2
0,1,Sun Blinds Decoration Inc.,Indl De Cuautitlan Sa Cv
1,2,Eih Ltd.,"Dongguan Wei Shi Plastic Product Co., Ltd."
2,3,Jsh Ltd. (Hk),Arab Shipbuilding And Repair Yard C
3,4,Better Industrial Ltd.,Farmacap Industria E Comercio Ltda
4,5,Equipos Inoxidables Del Norte Sa De Cv,Bel Inc.
...,...,...,...
213244,213245,Dyna Logistics Inc.,Jiangsu Easero Logistics Ltd.
213245,213246,Dsi,Kavalani & Sons W.L.L
213246,213247,Saint Gobain Abrasives (Shanghai),Zarabi
213247,213248,Covestro ( India) Private Ltd.,Terabytes India Private Ltd.


In [ ]:
train['is_duplicate'].unique()

array([0, 1])

In [ ]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
import pathlib
import pandas as pd

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
RS = 7345

# Загрузка данных

In [ ]:
train = pd.read_csv("train.csv", index_col="pair_id")
test = pd.read_csv("test.csv", index_col="pair_id")

In [ ]:
train.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [ ]:
test.head()

,name_1,name_2
pair_id,,
1,Sun Blinds Decoration Inc.,Indl De Cuautitlan Sa Cv
2,Eih Ltd.,"Dongguan Wei Shi Plastic Product Co., Ltd."
3,Jsh Ltd. (Hk),Arab Shipbuilding And Repair Yard C
4,Better Industrial Ltd.,Farmacap Industria E Comercio Ltda
5,Equipos Inoxidables Del Norte Sa De Cv,Bel Inc.


# Статистика таргета

In [ ]:
train.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [ ]:
train[train.is_duplicate==1].sample(15)

,name_1,name_2,is_duplicate
pair_id,,,
468048,INRECO,Inreco A/S,1
240795,Bridgestone Firestone Venezolana,Bridgestone Ncr,1
140871,Bridgestone Stargard Sp Zoo,"Bridgestone(Shenyang) Tire Co., Ltd.",1
474044,Youngnam Spinning,"Youngnam Spinning Co., Ltd.",1
325344,T.L. Squire,T.L. Squire & Co. Inc.,1
411877,SOPREMA BLONIE,SOPREMA SOURGES,1
287532,Exxonmobil Quimica Ltda,Exxonmobil Servicios Petroleros Mexico Sa De Cv,1
10505,Gestion Soprema Canada Inc.,SOPREMA SOURGES,1
283509,ВЕДАГ ГМБХ,VEDAG GmbH,1


In [ ]:
train[train.is_duplicate==0].sample(15)

,name_1,name_2,is_duplicate
pair_id,,,
104154,"Eia Co., Ltd.",Hisvima S.L,0
464074,Golden Trading Ltd.,Wld,0
262173,Truck Bus Industria E Comercio De Auto Pecas Ltda,Socer Rb Industries E Comercio Ltda,0
192322,Sto Corp,Ideal Combination,0
477878,Deybon International,The Bonnot Co.,0
426386,Giffin Industries S De Rl De Cv,Android Industries,0
185286,"Eosso Brothers, Inc",EUROBIT d.o.o.,0
320214,North American Wood Products,Chori America,0
103425,Henkel Industrie Ag No 3 5 Th Floor,Abc Industries,0


# Очистка данных

In [ ]:
!pip install pycountry

In [ ]:
import pycountry
import re

In [ ]:
countries = [country.name.lower() for country in pycountry.countries]

In [ ]:
train["name_1"] = train["name_1"].str.lower()
train["name_2"] = train["name_2"].str.lower()

test["name_1"] = test["name_1"].str.lower()
test["name_2"] = test["name_2"].str.lower()

In [ ]:
train[train.name_1.str.contains("gmbh")]

,name_1,name_2,is_duplicate
pair_id,,,
40,isoliererzeugnisse grobrohrsdorf gmbh,total rumunia,0
410,dortmunder gussasphalt gmbh & co. kg,bitoumina s.a.,0
440,celanese chemicals europe gmbh,inreco - emulsja sp. z o.o.,0
583,tremco illbruck group gmbh,wakol gmbh,0
681,rasco bitumentechnik gmbh,technpvinyl polymers india ltd.,0
...,...,...,...
497197,binne & sohn gmbh & co kg,buscher hoffmann,0
497306,mct gmbh & co. kg,valli zabban,0
497366,mct misch & compoundier technik gmbh,nknh,0


In [ ]:
legal_entities = ["ltd.", "co.", "inc.", "b.v.", "s.c.r.l.", "gmbh", "pvt."]

for entity in tqdm(legal_entities):
    train.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)
    test.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)

100%|██████████| 7/7 [00:14<00:00,  2.12s/it]


In [ ]:
train.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,iko industries,"enormous industrial trade,",0
2,apcotex industries,technocraft industries (india),0
3,"rishichem distributors,",dsa,0
4,powermax rubber factory,co. one,0
5,tress a/s,longyou industries park zhejiang,0


In [ ]:
test.head()

,name_1,name_2
pair_id,,
1,sun blinds decoration,indl de cuautitlan sa cv
2,eih,"dongguan wei shi plastic product,"
3,jsh(hk),arab shipbuilding and repair yard c
4,better industrial,farmacap industria eercio
5,equipos inoxidables del norte sa de cv,bel


In [ ]:
train.replace(re.compile(r"\s+\(.*\)"), "", inplace=True)
test.replace(re.compile(r"\s+\(.*\)"), "", inplace=True)

In [ ]:
for country in tqdm(countries):
    train.replace(re.compile(country), "", inplace=True)
    test.replace(re.compile(country), "", inplace=True)

100%|██████████| 249/249 [06:28<00:00,  1.56s/it]


In [ ]:
train.replace(re.compile(r"[^\w\s]"), "", inplace=True)
test.replace(re.compile(r"[^\w\s]"), "", inplace=True)

In [ ]:
train[:15]

,name_1,name_2,is_duplicate
pair_id,,,
1,iko industries,enormous industrial trade,0
2,apcotex industries,technocraft industries,0
3,rishichem distributors,dsa,0
4,powermax rubber factory,co one,0
5,tress as,longyou industries park zhejiang,0
6,wpt sa de cv,hopkins mfg de s de rl de cv,0
7,national bank of,action international,0
8,citibank korea,kyor,0
9,prettl electric del bajio sa de cv,bva,0


In [ ]:
test[:15]

,name_1,name_2
pair_id,,
1,sun blinds decoration,indl de cuautitlan sa cv
2,eih,dongguan wei shi plastic product
3,jshhk,arab shipbuilding and repair yard c
4,better industrial,farmacap industria eercio
5,equipos inoxidables del norte sa de cv,bel
6,afton chemical industria de aditivos,gigi industries
7,sumec international technology,advanced polymer technology
8,intermax imp exp,ningbo jiahui huamao imp exp
9,precision ice,uci


# Простые метрики сходства строк

Косинусово сходство

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances as pcd

clf = TfidfVectorizer()

clf.fit(train['name_1'] + " " + train['name_2'])

A = clf.transform(train['name_1'])
B = clf.transform(train['name_2'])

cosine = 1 - pcd(A, B)

train['tfidf_cosine'] = cosine

In [ ]:
train.sample(30)

,name_1,name_2,is_duplicate,tfidf_cosine
pair_id,,,,
51281,miyasaka polymer,kraton polymers us,0,-2.220446e-16
345075,valeo friction materials private,uci,0,0.000000e+00
323160,imperalum sociedadeercial de revestimentos e ...,porr ag,0,0.000000e+00
397761,atturo tire,sr tyres,0,0.000000e+00
294230,qingdao fest logistics,stalori s a,0,0.000000e+00
221535,qafelah al mahra trading est,ayyad al rashidi trading est,0,3.294840e-01
299961,monty industries sa de cv,grupo american industries sa de cv,0,3.366112e-01
451682,cdm systems,motherson sumi systems,0,2.048167e-01
306717,jas forwarding de sa de c,skf de sa de cv,0,3.470251e-01


In [ ]:
train.head()

,name_1,name_2,is_duplicate,tfidf_cosine
pair_id,,,,
1,iko industries,enormous industrial trade,0,1.110223e-16
2,apcotex industries,technocraft industries,0,1.311625e-01
3,rishichem distributors,dsa,0,0.000000e+00
4,powermax rubber factory,co one,0,0.000000e+00
5,tress as,longyou industries park zhejiang,0,-2.220446e-16


In [ ]:
test.head()

,name_1,name_2
pair_id,,
1,sun blinds decoration,indl de cuautitlan sa cv
2,eih,dongguan wei shi plastic product
3,jshhk,arab shipbuilding and repair yard c
4,better industrial,farmacap industria eercio
5,equipos inoxidables del norte sa de cv,bel


In [ ]:
clf = TfidfVectorizer()

clf.fit(test['name_1'] + " " + test['name_2'])

A = clf.transform(test['name_1'])
B = clf.transform(test['name_2'])

cosine = 1 - pcd(A, B)

test['tfidf_cosine'] = cosine

In [ ]:
test[40:60]

,name_1,name_2,tfidf_cosine
pair_id,,,
41,continental airtrol,continental powertrain usa llc,2.989800e-01
42,guangxing hardware plastics,ga logisticsas,-2.220446e-16
43,gap,anchicos tecnologia e ingenieria sas,0.000000e+00
44,biscom,lae biscuits,0.000000e+00
45,olympia industries,balkrishna industria,0.000000e+00
46,maniarporation,minipak sas entrega en lugar de,0.000000e+00
47,camelot logistics,logistica essa sa de cv,0.000000e+00
48,orion plastics,evolution logistics,0.000000e+00
49,banglore heavy equipments,aim international,1.110223e-16


# Базовая модель

In [ ]:
FTS=["tfidf_cosine"]
THR = 0.9

In [ ]:
split = StratifiedShuffleSplit(1, train_size=0.8, random_state=RS)
tridx, cvidx = list(split.split(train[FTS], train["is_duplicate"]))[0]

###Классификаторы

Random Forest

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators = 1000)
rf.fit(train.iloc[tridx][FTS].values,
          train.iloc[tridx]["is_duplicate"].values)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
tr_preds_rf = rf.predict(train.iloc[tridx][FTS].values)
cv_preds_rf = rf.predict(train.iloc[cvidx][FTS].values)

In [ ]:
print(f1_score(train.iloc[tridx]["is_duplicate"].values, tr_preds_rf))
print(f1_score(train.iloc[cvidx]["is_duplicate"].values, cv_preds_rf))

0.900147383935151
0.6764091858037578


Extremely Randomized Trees

In [ ]:
ert = ensemble.ExtraTreesClassifier(n_estimators = 1000)
ert.fit(train.iloc[tridx][FTS].values,
          train.iloc[tridx]["is_duplicate"].values)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
tr_preds = ert.predict(train.iloc[tridx][FTS].values)
cv_preds = ert.predict(train.iloc[cvidx][FTS].values)

In [ ]:
print(f1_score(train.iloc[tridx]["is_duplicate"].values, tr_preds))
print(f1_score(train.iloc[cvidx]["is_duplicate"].values, cv_preds))

0.900754925428098
0.6773519163763065


GradientBoostingClassifier

In [ ]:

gbt = ensemble.GradientBoostingClassifier(n_estimators = 1000)
#gbt.fit(gbt = ensemble.GradientBoostingClassifier(n_estimators = 1000)
gbt.fit(train.iloc[tridx][FTS].values,
          train.iloc[tridx]["is_duplicate"].values)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
y_train_pred = gbt.predict(train.iloc[tridx][FTS].values)
y_test_pred = gbt.predict(train.iloc[cvidx][FTS].values)

In [ ]:
print(f1_score(train.iloc[tridx]["is_duplicate"].values, y_train_pred))
print(f1_score(train.iloc[cvidx]["is_duplicate"].values, y_test_pred))

0.5934372818245287
0.4735376044568246


# Генерация сабмита

In [ ]:
sample_sub = pd.read_csv("sample_submission (2).csv", index_col="pair_id")

In [ ]:
sample_sub

,is_duplicate
pair_id,
1,1
2,1
3,1
4,1
5,1
...,...
213245,1
213246,1
213247,1


In [ ]:
sample_sub["is_duplicate"] = ert.predict(test[FTS].values)


In [ ]:
sample_sub.is_duplicate.value_counts()

0    212387
1       862
Name: is_duplicate, dtype: int64

In [ ]:
sample_sub.to_csv("submission_cos_ert.csv")

In [ ]:
sample_sub["is_duplicate"] = rf.predict(test[FTS].values)

In [ ]:
sample_sub.is_duplicate.value_counts()

0    212390
1       859
Name: is_duplicate, dtype: int64

In [ ]:
sample_sub.to_csv("submission_cos_rf.csv")